# MASKRCNN DEMO ÖRNEĞİ

## Başlangıç

Önceden eğittiğimiz modelimiz sayesinde seçtiğimiz resimler üzerinde segmentasyon işlemini gerçekleştirip nesneleri tanıyan bir örnek yapıyoruz. 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
# Projenin root dosya yolu
ROOT_DIR = os.path.abspath("../")
# Mask RCNN'i import ediyoruz
sys.path.append(ROOT_DIR) 
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# COCO konfigürasyonlarını entegre ediyoruz
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/")) 
import coco
%matplotlib inline
# loglarımızı kaydetmek için bir dosya yolu oluşturuyoruz
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
# Resimlerimizin dosya yollarını belirtiyoruz
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

## Konfigürasyonlar

MS-COCO veri kümesi üzerinde eğitilmiş bir model kullanacağız. Bu modelin konfigürasyonları, coco.py'deki CocoConfig sınıfındadır. 

In [ ]:
class InferenceConfig(coco.CocoConfig):
 # Çıkarım yapacağımız için batch boyutunu 1 olarak ayarladık
 # Her seferinde bir resim işlenir. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()
config.display()

## Modeli Oluşturma ve Eğitilmiş Ağırlıkları Ekleme

In [ ]:
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
# MS-COCO üzerinden eğitilmiş ağırlıklar eklendi
model.load_weights(COCO_MODEL_PATH, by_name=True)

## Sınıf İsimleri

Model, nesneleri sınıflandırır ve her bir sınıfı tanımlayan tamsayı değeri olan sınıf kimliklerini döndürür. Bazı veri kümeleri diğer sınıflara tamsayı değerleri atarken bazıları vermez. Örneğin, MS-COCO veri kümesinde, 'person' sınıfı 1 ve 'teddy bear' 88'dir.
Kimlikler genellikle sıralıdır, ancak her zaman değil. Örneğin, COCO veri kümesinde sınıf kimlikleri 70 ve 72 ile ilişkili sınıflar var, ancak 71 değil. Tutarlılığı iyileştirmek ve aynı anda birden çok kaynaktan gelen verileri taramayı desteklemek için Veri Kümesi sınıfımız, her birine kendi sıralı tam sayı kimlikleri atar. sınıf. Örneğin, Veri Kümesi sınıfımızı kullanarak COCO veri kümesini yüklerseniz, 'kişi' sınıfı, sınıf kimliği = 1 (aynı COCO gibi) ve 'teddy bear' sınıfı 78 alır(COCO'dan farklı).
Sınıf adlarının listesini almak için, veri kümesini yükler ve ardından class_names özelliğini aşağıdaki gibi kullanırız.


### COCO datasetinin yüklenmesi
dataset = coco.CocoDataset()
dataset.load_coco(COCO_DIR, "train")
dataset.prepare()
### Sınıf isimlerinin ekranda gösterilmesi
print(dataset.class_names)


In [ ]:
# COCO Class isimleri
#Listedeki classın index numarası o classın IDsidir. 
#Örneğin, teddy bear sınıfının kimliğini almak için şu kullanılır: class_names.index ('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
 'bus', 'train', 'truck', 'boat', 'traffic light',
 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
 'kite', 'baseball bat', 'baseball glove', 'skateboard',
 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
 'teddy bear', 'hair drier', 'toothbrush']

## Nesne Tanımayı Başlatma

Bu kısımda resimlerimizin bulunduğu images klasöründen rastgele bir resim seçerek o resimdeki nesneleri tanımasını bekliyoruz.


In [ ]:
file_names = next(os.walk(IMAGE_DIR))[2]
#Seçtiğimiz resmin dosya yolunu yazıyoruz.
filename=os.path.join(IMAGE_DIR, '12283150_12d37e6389_z.jpg')
image = skimage.io.imread(filename)
results = model.detect([image], verbose=1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
 class_names, r['scores'])